In [2]:
import pandas as pd
import os
import yaml # Necesario para leer archivos .dvc (YAML)
import mlflow
import mlflow.data
import mlflow.sklearn

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

# --- IMPORTACIONES DE MÉTRICAS ---
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    precision_score,
    recall_score,
    f1_score
)

# --- CONFIGURACIÓN DE RUTAS ---
DATASET_PATH = '../data/interim/student_interim_clean_for_model_2.csv' 
DATASET_NAME = 'student_entry_clean'

# --- 1. CARGA DE DATOS (Mismos pasos) ---
try:
    df = pd.read_csv(DATASET_PATH)
    print(f"Dataset cargado desde: {DATASET_PATH}")
except FileNotFoundError:
    print(f"ERROR: Archivo no encontrado en {DATASET_PATH}. Revisa la ruta.")
    exit()

# --- 2. LÓGICA DVC: OBTENER EL HASH (Mismos pasos) ---
dvc_digest = None
dvc_file_path = DATASET_PATH + ".dvc"
if os.path.exists(dvc_file_path):
    try:
        with open(dvc_file_path, 'r') as f:
            dvc_data = yaml.safe_load(f)
        if 'outs' in dvc_data and dvc_data['outs']:
            dvc_digest = dvc_data['outs'][0].get('md5') 
            if not dvc_digest:
                dvc_digest = dvc_data['outs'][0].get('checksum') 
        print(f"DVC Digest encontrado: {dvc_digest}")
    except Exception as e:
        print(f"ADVERTENCIA: No se pudo leer el archivo DVC. Error: {e}")

# --- 3. PREPARACIÓN DE DATOS Y SPLIT (Mismos pasos) ---
X = df.drop(columns=['Performance']) 
y = df['Performance']
cat_cols = ['Gender','Caste','coaching','time','Class_ten_education','twelve_education','medium','Class_ X_Percentage','Class_XII_Percentage','Father_occupation','Mother_occupation']

le = LabelEncoder()
y_enc = le.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y_enc, test_size=0.2, stratify=y_enc, random_state=42)


# ---------------------------------------------------------------------
# --- 4. CONFIGURACIÓN DEL PIPELINE BASE Y LA GRILLA DE BÚSQUEDA ---
# ---------------------------------------------------------------------

# Pipeline base (solo preprocesamiento y clasificador, sin parámetros aún)
preprocessor = ColumnTransformer(
    transformers=[('ohe', OneHotEncoder(handle_unknown='ignore'), cat_cols)],
    remainder='drop'
)
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor), 
    ('clf', RandomForestClassifier(random_state=888))
])

# Grilla de Parámetros para GridSearchCV (¡NUEVO!)
# Nota: Los nombres deben coincidir con la convención del pipeline: <nombre_paso>__<parametro>
param_grid = {
    'clf__n_estimators': [50, 100, 200],  # Cantidad de árboles
    'clf__max_depth': [5, 10, None],      # Profundidad máxima
    'clf__min_samples_split': [5, 10]     # Muestras mínimas para dividir
}

# Configuración del Grid Search (Usando 'f1_weighted' como métrica principal para multiclase)
grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    scoring='f1_weighted',  # Métrica de optimización
    cv=5,                   # 5-fold cross-validation
    verbose=2,
    n_jobs=1
)

# --- 5. ENTRENAMIENTO Y PREDICCIONES (Ejecución del Grid Search) ---
print("\nIniciando Grid Search...")
grid_search.fit(X_train, y_train)
print("Grid Search finalizado.")


# ---------------------------------------------------------------------
# --- 6. REGISTRO DE TODAS LAS CORRIDAS EN MLFLOW ---
# ---------------------------------------------------------------------

mlflow.set_tracking_uri("http://127.0.0.1:5001")
EXPERIMENT_NAME = "randomforest_GridSearch" # Nuevo nombre de experimento para la búsqueda
mlflow.set_experiment(EXPERIMENT_NAME)


# Iterar sobre CADA resultado de CV (Cross-Validation)
for i, (mean_score, std_score, params) in enumerate(zip(
    grid_search.cv_results_['mean_test_score'],
    grid_search.cv_results_['std_test_score'],
    grid_search.cv_results_['params']
)):
    
    # 1. Iniciar una nueva corrida para cada combinación de hiperparámetros
    with mlflow.start_run(run_name=f"run_{i+1}_RF_GridSearch", nested=True) as run:
        print(f"Registrando corrida {i+1} con parámetros: {params}")

        # Reconstruir el pipeline para este conjunto de parámetros (para log_model)
        current_pipeline = Pipeline(steps=[
            ('preprocessor', preprocessor), 
            ('clf', RandomForestClassifier(random_state=888, **params))
        ])
        
        # Entrenar el modelo con el conjunto de entrenamiento COMPLETO usando los mejores parámetros
        # NOTA: En la práctica se usa grid_search.best_estimator_ o se reentrena. 
        # Aquí, para fines de registro de TODAS las combinaciones, reentrenamos con la combinación actual
        current_pipeline.fit(X_train, y_train)
        
        # --- Predicciones y Cálculo de Métricas (en el conjunto de prueba) ---
        y_pred_test = current_pipeline.predict(X_test)
        
        # Calcular Métricas (Incluyendo las de promedio)
        acc_test = accuracy_score(y_test, y_pred_test)
        f1_micro = f1_score(y_test, y_pred_test, average='micro')
        f1_macro = f1_score(y_test, y_pred_test, average='macro')
        f1_weighted = f1_score(y_test, y_pred_test, average='weighted')
        report_text = classification_report(y_test, y_pred_test, target_names=le.classes_)
        
        metrics = {
            "cv_f1_weighted_mean": mean_score, # Métrica de CV (crucial para GridSearch)
            "cv_f1_weighted_std": std_score,
            "test_acc": acc_test,
            "test_f1_weighted": f1_weighted,
            "test_f1_macro": f1_macro,
        }
        
        # --- Registro en MLflow ---
        
        # 1. Registrar hiperparámetros
        mlflow.log_params(params)
        
        # 2. Registrar métricas
        mlflow.log_metrics(metrics)
        
        # 3. Guardar modelo
        mlflow.sklearn.log_model(current_pipeline, "random_forest_pipeline")
        
        # 4. Registrar Dataset (solo se hace una vez por experimentación, pero lo incluimos)
        mlflow_dataset = mlflow.data.from_pandas(
            df=df, source=DATASET_PATH, targets=y.name, name=DATASET_NAME, digest=dvc_digest)
        mlflow.log_input(mlflow_dataset, context="training") 

        # 5. Registrar Classification Report (como artefacto)
        temp_report_path = f"classification_report_run_{i+1}.txt"
        with open(temp_report_path, "w") as f:
            f.write(report_text)
        mlflow.log_artifact(temp_report_path, artifact_path="report")
        os.remove(temp_report_path)

        # 6. Registrar el modelo con el mejor rendimiento (opcional, solo si el F1 de CV es el mejor hasta ahora)
        if mean_score == grid_search.best_score_:
             print(f"!!! Este es el mejor modelo (F1-Weighted CV: {mean_score:.4f}) !!!")
             mlflow.set_tag("best_run", "True")


print("\n--- RESUMEN FINAL DE GRID SEARCH ---")
print(f"El mejor F1-Weighted (CV) es: {grid_search.best_score_:.4f}")
print(f"Los mejores parámetros son: {grid_search.best_params_}")

Dataset cargado desde: ../data/interim/student_interim_clean_for_model_2.csv
DVC Digest encontrado: 76db7197326a5942db9c5b100349f69b

Iniciando Grid Search...
Fitting 5 folds for each of 18 candidates, totalling 90 fits
[CV] END clf__max_depth=5, clf__min_samples_split=5, clf__n_estimators=50; total time=   0.0s
[CV] END clf__max_depth=5, clf__min_samples_split=5, clf__n_estimators=50; total time=   0.0s
[CV] END clf__max_depth=5, clf__min_samples_split=5, clf__n_estimators=50; total time=   0.0s
[CV] END clf__max_depth=5, clf__min_samples_split=5, clf__n_estimators=50; total time=   0.0s
[CV] END clf__max_depth=5, clf__min_samples_split=5, clf__n_estimators=50; total time=   0.0s
[CV] END clf__max_depth=5, clf__min_samples_split=5, clf__n_estimators=100; total time=   0.0s
[CV] END clf__max_depth=5, clf__min_samples_split=5, clf__n_estimators=100; total time=   0.1s
[CV] END clf__max_depth=5, clf__min_samples_split=5, clf__n_estimators=100; total time=   0.1s
[CV] END clf__max_depth=5

2025/11/02 17:57:48 INFO mlflow.tracking.fluent: Experiment with name 'randomforest_GridSearch' does not exist. Creating a new experiment.


[CV] END clf__max_depth=None, clf__min_samples_split=10, clf__n_estimators=200; total time=   0.1s
Grid Search finalizado.


2025/11/02 17:57:48 INFO mlflow.tracking._tracking_service.client: 🏃 View run run_1_RF_GridSearch at: http://127.0.0.1:5001/#/experiments/993790272377427900/runs/1e7d91931a594af2852c01f0546c487d.
2025/11/02 17:57:48 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5001/#/experiments/993790272377427900.


Registrando corrida 1 con parámetros: {'clf__max_depth': 5, 'clf__min_samples_split': 5, 'clf__n_estimators': 50}


TypeError: RandomForestClassifier.__init__() got an unexpected keyword argument 'clf__max_depth'

In [ ]:
#FUNCIONANDO
import mlflow
import mlflow.data
import mlflow.sklearn

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

# --- IMPORTACIONES DE MÉTRICAS ---
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    precision_score,
    recall_score,
    f1_score
)

# --- CONFIGURACIÓN DE RUTAS ---
DATASET_PATH = '../data/interim/student_interim_clean_for_model_2.csv' 
DATASET_NAME = 'student_entry_clean'

# --- 1. CARGA DE DATOS ---
try:
    df = pd.read_csv(DATASET_PATH)
    print(f"Dataset cargado desde: {DATASET_PATH}")
except FileNotFoundError:
    print(f"ERROR: Archivo no encontrado en {DATASET_PATH}. Revisa la ruta.")
    exit()

# --- 2. LÓGICA DVC: OBTENER EL HASH (DIGEST) ---
dvc_digest = None
dvc_file_path = DATASET_PATH + ".dvc"
if os.path.exists(dvc_file_path):
    try:
        with open(dvc_file_path, 'r') as f:
            dvc_data = yaml.safe_load(f)
        if 'outs' in dvc_data and dvc_data['outs']:
            dvc_digest = dvc_data['outs'][0].get('md5') 
            if not dvc_digest:
                dvc_digest = dvc_data['outs'][0].get('checksum') 
        print(f"DVC Digest encontrado: {dvc_digest}")
    except Exception as e:
        print(f"ADVERTENCIA: No se pudo leer el archivo DVC. Error: {e}")

# --- 3. PREPARACIÓN DE DATOS Y SPLIT ---
X = df.drop(columns=['Performance']) 
y = df['Performance']
cat_cols = ['Gender','Caste','coaching','time','Class_ten_education','twelve_education','medium','Class_ X_Percentage','Class_XII_Percentage','Father_occupation','Mother_occupation']

le = LabelEncoder()
y_enc = le.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y_enc, test_size=0.2, stratify=y_enc, random_state=42)

# ---------------------------------------------------------------------
# --- 4. CONFIGURACIÓN DEL PIPELINE BASE Y LA GRILLA DE BÚSQUEDA ---
# ---------------------------------------------------------------------

# Pipeline base (solo preprocesamiento y clasificador, sin parámetros aún)
preprocessor = ColumnTransformer(
    transformers=[('ohe', OneHotEncoder(handle_unknown='ignore'), cat_cols)],
    remainder='drop'
)
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor), 
    ('clf', RandomForestClassifier(random_state=888))
])

# Grilla de Parámetros para GridSearchCV
param_grid = {
    'clf__n_estimators': [50, 100, 200],  # Cantidad de árboles
    'clf__max_depth': [5, 10, None],      # Profundidad máxima
    'clf__min_samples_split': [5, 10]     # Muestras mínimas para dividir
}

# Configuración del Grid Search
grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    scoring='f1_weighted',
    cv=5,                   
    verbose=2,
    n_jobs=1  # Corregido a 1 para evitar errores de paralelización/pickling
)

# --- 5. ENTRENAMIENTO Y PREDICCIONES (Ejecución del Grid Search) ---
print("\nIniciando Grid Search...")
grid_search.fit(X_train, y_train)
print("Grid Search finalizado.")


# ---------------------------------------------------------------------
# --- 6. REGISTRO DE TODAS LAS CORRIDAS EN MLFLOW ---
# ---------------------------------------------------------------------

MLFLOW_MODEL_NAME = "Student_Performance_RF_Model" # Nombre para el Model Registry

mlflow.set_tracking_uri("http://127.0.0.1:5001")
EXPERIMENT_NAME = "randomforest_GridSearch"
mlflow.set_experiment(EXPERIMENT_NAME)

best_run_id = None
best_model_uri = None

# Iterar sobre CADA resultado de CV (Cross-Validation)
for i, (mean_score, std_score, params) in enumerate(zip(
    grid_search.cv_results_['mean_test_score'],
    grid_search.cv_results_['std_test_score'],
    grid_search.cv_results_['params']
)):
    
    with mlflow.start_run(run_name=f"run_{i+1}_RF_GridSearch", nested=True) as run:
        print(f"Registrando corrida {i+1} con parámetros: {params}")

        # --- CORRECCIÓN: LIMPIEZA DE PARÁMETROS ---
        # Remueve el prefijo 'clf__' para que RandomForestClassifier lo acepte
        clf_params = {k.replace('clf__', ''): v for k, v in params.items()}
        # ------------------------------------------
        
        # Reconstruir el pipeline con los parámetros actuales
        current_pipeline = Pipeline(steps=[
            ('preprocessor', preprocessor), 
            ('clf', RandomForestClassifier(random_state=888, **clf_params)) 
        ])
        
        # Reentrenar el modelo con el conjunto de entrenamiento COMPLETO para este registro
        current_pipeline.fit(X_train, y_train)
        
        # --- Cálculo de Métricas ---
        y_pred_test = current_pipeline.predict(X_test)
        
        acc_test = accuracy_score(y_test, y_pred_test)
        f1_micro = f1_score(y_test, y_pred_test, average='micro')
        f1_macro = f1_score(y_test, y_pred_test, average='macro')
        f1_weighted = f1_score(y_test, y_pred_test, average='weighted')
        report_text = classification_report(y_test, y_pred_test, target_names=le.classes_)
        
        metrics = {
            "cv_f1_weighted_mean": mean_score, 
            "cv_f1_weighted_std": std_score,
            "test_acc": acc_test,
            "test_f1_weighted": f1_weighted,
            "test_f1_macro": f1_macro,
        }
        
        # --- Registro en MLflow ---
        mlflow.log_params(params)
        mlflow.log_metrics(metrics)
        
        # 3. Guardar modelo (artefacto)
        model_info = mlflow.sklearn.log_model(current_pipeline, "random_forest_pipeline")
        
        # 4. Registrar Dataset y Reporte (artefactos)
        mlflow_dataset = mlflow.data.from_pandas(
            df=df, source=DATASET_PATH, targets=y.name, name=DATASET_NAME, digest=dvc_digest)
        mlflow.log_input(mlflow_dataset, context="training") 

        temp_report_path = f"classification_report_run_{i+1}.txt"
        with open(temp_report_path, "w") as f:
            f.write(report_text)
        mlflow.log_artifact(temp_report_path, artifact_path="report")
        os.remove(temp_report_path)

        # --- Identificación del Mejor Modelo ---
        if mean_score == grid_search.best_score_:
             print(f"!!! Este es el mejor modelo (F1-Weighted CV: {mean_score:.4f}) !!!")
             mlflow.set_tag("best_run", "True")
             best_run_id = run.info.run_id
             best_model_uri = model_info.model_uri # URI para el Model Registry

print("\n--- RESUMEN FINAL DE GRID SEARCH ---")
print(f"El mejor F1-Weighted (CV) es: {grid_search.best_score_:.4f}")
print(f"Los mejores parámetros son: {grid_search.best_params_}")


# ---------------------------------------------------------------------
# --- 7. REGISTRO FINAL DEL MEJOR MODELO EN MODEL REGISTRY ---
# ---------------------------------------------------------------------

if best_model_uri:
    print(f"\n--- REGISTRANDO EL MEJOR MODELO EN REGISTRY ---")
    try:
        # Registra el modelo como versión 'initial' en el Registry
        model_version = mlflow.register_model(
            model_uri=best_model_uri,
            name=MLFLOW_MODEL_NAME
        )
        print(f"[OK] Modelo '{MLFLOW_MODEL_NAME}' registrado como versión: {model_version.version}")
        print(f"URI del Modelo: {best_model_uri}")
    except Exception as e:
        print(f"[ERROR] No se pudo registrar el modelo en el Registry: {e}")

Dataset cargado desde: ../data/interim/student_interim_clean_for_model_2.csv
DVC Digest encontrado: 76db7197326a5942db9c5b100349f69b

Iniciando Grid Search...
Fitting 5 folds for each of 18 candidates, totalling 90 fits
[CV] END clf__max_depth=5, clf__min_samples_split=5, clf__n_estimators=50; total time=   0.0s
[CV] END clf__max_depth=5, clf__min_samples_split=5, clf__n_estimators=50; total time=   0.0s
[CV] END clf__max_depth=5, clf__min_samples_split=5, clf__n_estimators=50; total time=   0.0s
[CV] END clf__max_depth=5, clf__min_samples_split=5, clf__n_estimators=50; total time=   0.0s
[CV] END clf__max_depth=5, clf__min_samples_split=5, clf__n_estimators=50; total time=   0.0s
[CV] END clf__max_depth=5, clf__min_samples_split=5, clf__n_estimators=100; total time=   0.0s
[CV] END clf__max_depth=5, clf__min_samples_split=5, clf__n_estimators=100; total time=   0.0s
[CV] END clf__max_depth=5, clf__min_samples_split=5, clf__n_estimators=100; total time=   0.0s
[CV] END clf__max_depth=5

c:\Users\Anuar\Documents\Maestria Inteligencia artificial\MLOPs Prueba\Cookie cutter 2\proyecto_mlops_equipo_56\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Anuar\Documents\Maestria Inteligencia artificial\MLOPs Prueba\Cookie cutter 2\proyecto_mlops_equipo_56\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Anuar\Documents\Maestria Inteligencia artificial\MLOPs Prueba\Cookie cutter 2\proyecto_mlops_equipo_56\.venv\Lib\site-packages\sklearn\metrics\_classification.py:153

Registrando corrida 2 con parámetros: {'clf__max_depth': 5, 'clf__min_samples_split': 5, 'clf__n_estimators': 100}


2025/11/02 18:07:01 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\Anuar\AppData\Local\Temp\tmp9mkn4wsa\model\model.pkl, flavor: sklearn). Fall back to return ['scikit-learn==1.5.2', 'cloudpickle==2.2.1']. Set logging level to DEBUG to see the full traceback. 
2025/11/02 18:07:01 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
c:\Users\Anuar\Documents\Maestria Inteligencia artificial\MLOPs Prueba\Cookie cutter 2\proyecto_mlops_equipo_56\.venv\Lib\site-packages\mlflow\data\dataset_source_registry.py:150: UserWarning: Failed to determine whether UCVolumeDatasetSource can resolve source information for '../data/interim/student_interim_clean_for_model_2.csv'. Exception: 
  return _dataset_source_registry.resolve(
c:\Users\Anuar\Documents\Maestria Inteligencia artificial\MLOPs Prueba\Cookie cutter 2\proyecto_mlops_equi

Registrando corrida 3 con parámetros: {'clf__max_depth': 5, 'clf__min_samples_split': 5, 'clf__n_estimators': 200}


...

Registrando corrida 10 con parámetros: {'clf__max_depth': 10, 'clf__min_samples_split': 10, 'clf__n_estimators': 50}


2025/11/02 18:25:52 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\Anuar\AppData\Local\Temp\tmpu8_qrykd\model\model.pkl, flavor: sklearn). Fall back to return ['scikit-learn==1.5.2', 'cloudpickle==2.2.1']. Set logging level to DEBUG to see the full traceback. 
2025/11/02 18:25:53 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
c:\Users\Anuar\Documents\Maestria Inteligencia artificial\MLOPs Prueba\Cookie cutter 2\proyecto_mlops_equipo_56\.venv\Lib\site-packages\mlflow\data\dataset_source_registry.py:150: UserWarning: Failed to determine whether UCVolumeDatasetSource can resolve source information for '../data/interim/student_interim_clean_for_model_2.csv'. Exception: 
  return _dataset_source_registry.resolve(
c:\Users\Anuar\Documents\Maestria Inteligencia artificial\MLOPs Prueba\Cookie cutter 2\proyecto_mlops_equi

!!! Este es el mejor modelo (F1-Weighted CV: 0.5031) !!!
Registrando corrida 11 con parámetros: {'clf__max_depth': 10, 'clf__min_samples_split': 10, 'clf__n_estimators': 100}


c:\Users\Anuar\Documents\Maestria Inteligencia artificial\MLOPs Prueba\Cookie cutter 2\proyecto_mlops_equipo_56\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Anuar\Documents\Maestria Inteligencia artificial\MLOPs Prueba\Cookie cutter 2\proyecto_mlops_equipo_56\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Anuar\Documents\Maestria Inteligencia artificial\MLOPs Prueba\Cookie cutter 2\proyecto_mlops_equipo_56\.venv\Lib\site-packages\sklearn\metrics\_classification.py:153

: 